Demo from https://colab.research.google.com/github/facebookresearch/moco/blob/colab-notebook/colab/moco_cifar10_demo.ipynb

In [3]:
gpu_info = !nvidia-smi -i 0
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

from datetime import datetime
from functools import partial
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchvision.models import resnet
from tqdm import tqdm
import argparse
import json
import math
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

Thu Jul 29 16:45:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3060    Off  | 00000000:08:00.0  On |                  N/A |
|  0%   46C    P8    18W / 170W |    812MiB / 12050MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# Parameters 
# Why would I want to use cmd arguments??

args = argparse.Namespace()

args.arch='resnet18' 
args.batch_size=512 
args.bn_splits=8 
args.cos=True 
args.epochs=200 
args.knn_k=200 
args.knn_t=0.1 
args.lr=0.06 
args.moco_dim=128 
args.moco_k=4096 
args.moco_m=0.99 
args.moco_t=0.1 
args.results_dir='./cache-2021-07-29-03-55-17-moco' 
args.resume='' 
args.schedule=[] 
args.symmetric=False 
args.wd=0.0005

## Data loader 

Need train / memory / test data sets

train set returns a pair of images (same image, differently transformed)

In [17]:
from typing import Any, Callable, Optional, Tuple
from torchvision.datasets import VisionDataset

class TonemapImageDataset(VisionDataset):
    def __init__(self, 
                 data_array, 
                 labels: Optional = None, 
                 train: bool=True,
                 transform: Optional[Callable] = None,
                 target_transform: Optional[Callable] = None,):
        self._array = data_array
        self._good_gids = np.array([gal['img_name'] for gal in data_array])
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self) -> int:
        return len(self.img_labels)
    
    def __getitem__(self, idx: int) -> Tuple[Any, Any]:
        """
        For super
        """
        image, _segmap, weight = self._array[idx]
        image[_segmap] = np.nan # Is it OK to have nan?
        image[image < 0] = 0
        
        label = self.labels.iloc[idx]
        return image
    
class TonemapImageDatasetPair(TonemapImageDataset):
    """
    returns two differently (randomly) transformed version of an image.
    """
    def __getitem__(self, idx):
        image, segmap, weight = self._array[idx]
        
        image[_segmap] = np.nan # Is it OK to have nan?
        image[image < 0] = 0
        
        label = self.labels.iloc[idx]
        
        if self.transform is not None:
            im_1 = self.transform(image) # random transform. 
            im_2 = self.transform(image)
        return im_1, im_2
        
        
        
        

In [18]:
import pickle
import numpy as np

fn = "../../bf_data/Nair_and_Abraham_2010/all_gals.pickle"
all_gals = pickle.load(open(fn, "rb"))

all_gals = all_gals[1:] # Why the first galaxy image is NaN?

good_gids = np.array([gal['img_name'] for gal in all_gals])

In [16]:
from astrobf.utils.misc import load_Nair
cat_data = load_Nair('../../bf_data/Nair_and_Abraham_2010/catalog/table2.dat')
# pd dataframe

cat = cat_data[cat_data['ID'].isin(good_gids)]

In [20]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

# data prepare
train_data = TonemapImageDatasetPair(all_gals, train=True, transform=train_transform)
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=16, pin_memory=True, drop_last=True)

memory_data = TonemapImageDataset(all_gals, train=True, transform=test_transform)
memory_loader = DataLoader(memory_data, batch_size=args.batch_size, shuffle=False, num_workers=16, pin_memory=True)

test_data = TonemapImageDataset(all_gals, train=False, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=args.batch_size, shuffle=False, num_workers=16, pin_memory=True)

AttributeError: 'TonemapImageDatasetPair' object has no attribute 'img_labels'